In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def map_ddc(ddc_code):
    mod_ddc_code = ''
    #print(ddc_code)
    if ddc_code:
        ddc_code = int(ddc_code)
        if ddc_code < 100:
            mod_ddc_code=0
        elif 100 <= ddc_code < 200:
            mod_ddc_code=100
        elif 200 <= ddc_code < 300:
            mod_ddc_code=200
        elif 300 <= ddc_code < 400:
            mod_ddc_code=300
        elif 400 <= ddc_code < 500:
            mod_ddc_code=400
        elif 500 <= ddc_code < 600:
            mod_ddc_code=500
        elif 600 <= ddc_code < 700:
            mod_ddc_code=600
        elif 700 <= ddc_code < 800:
            mod_ddc_code=700
        elif 800 <= ddc_code < 900:
            mod_ddc_code=800
        elif 900 <= ddc_code < 1000:
            mod_ddc_code=900
    return mod_ddc_code 

In [3]:
folder="../dataset_v5/v5_2/v5_2_2/"

In [4]:
edges = pd.read_csv(folder+'final_shodhganga_dataset_v5_2_7_2.csv')

In [5]:
edges["ddc_code"] = edges["ddc_code"].apply(lambda x : eval(x)[-1] if len(eval(x))>0 else '')

In [6]:
edges['ddc_code'].head()

0    580
1    300
2    300
3    300
4    300
Name: ddc_code, dtype: object

In [7]:
edges['ddc_code'] = edges['ddc_code'].apply(lambda x : map_ddc(x))

In [8]:
edges.rename(columns= {'advId':'source', 'resId':'target', 'new_date_awarded':'timestamp' }, inplace=True)

In [9]:
edges = edges[['source','target','timestamp','ddc_code','publisher_institution','instituteId']]

In [10]:
edges = edges.replace({'NaT': ''})

In [11]:
edges.columns

Index(['source', 'target', 'timestamp', 'ddc_code', 'publisher_institution',
       'instituteId'],
      dtype='object')

In [12]:
nodes = pd.read_csv(folder+'index_file2_v5_2_7_2.csv')

In [13]:
nodes.columns

Index(['rid', 'names'], dtype='object')

In [14]:
nodes['names'] = nodes['names'].apply(lambda x : x.split("|")[0])

In [15]:
#nodes['names'].sample(10)

In [16]:
nodes_names = dict(zip(nodes["rid"],nodes["names"]))

In [17]:
nodes["inst"] = nodes['rid'].apply(lambda x : set(edges[(edges['source']==x) | (edges['target']==x)]['publisher_institution'].values))

In [18]:
nodes["inst_select"] = nodes["inst"].apply(lambda x : list(x)[0])

In [19]:
nodes["ddc_code"] = nodes['rid'].apply(lambda x : list(set(edges[(edges['source']==x) | (edges['target']==x)]['ddc_code'].values)))

In [20]:
#nodes["ddc_code"]

In [20]:
nodes['ddc_count'] = nodes["ddc_code"].apply(lambda x: len(x))

In [21]:
nodes['ddc_select'] = nodes["ddc_code"].apply(lambda x: sorted([a for a in x if a!=""]) if len(x) > 0 else "" )

In [22]:
nodes['ddc_select'] = nodes["ddc_select"].apply(lambda x: x[-1] if len(x) > 0 else '')

In [23]:
#nodes['ddc_select_gen'] = nodes['ddc_select'].apply(lambda x : map_ddc(x))

In [24]:
nodes_inst =  dict(zip(nodes["rid"],nodes["inst_select"]))

In [25]:
nodes_ddc  =  dict(zip(nodes["rid"],nodes["ddc_select"]))

In [27]:
# edges['new_date_awarded']=edges['new_date_awarded'].astype(str)

In [28]:
graph = nx.convert_matrix.from_pandas_edgelist(edges, 'source','target','timestamp', create_using=nx.DiGraph())

In [29]:
nx.set_node_attributes(graph, values = nodes_names, name='name')

In [30]:
nx.set_node_attributes(graph, values = nodes_inst, name='inst')

In [31]:
nx.set_node_attributes(graph, values = nodes_ddc, name='ddc')

In [ ]:
#nx.write_gml(graph, "shodhganga_graph_v5_2_6-1.gml")

In [32]:
nx.readwrite.graphml.write_graphml(graph, "shodhganga_graph_v5_2_7_2.graphml")

In [33]:
nodes.to_csv("shodhganga_graph_nodes_v5_2_7_2.csv", index=False)

In [34]:
edges.columns

Index(['source', 'target', 'timestamp', 'ddc_code', 'publisher_institution',
       'instituteId'],
      dtype='object')

In [35]:
edges.to_csv("shodhganga_graph_edges_v5_2_7_2.csv", index=False)

In [36]:
graph.nodes[200]

{'name': 'Brar, Jagjit Singh', 'inst': 'Panjab University', 'ddc': 300}

In [37]:
edges.columns

Index(['source', 'target', 'timestamp', 'ddc_code', 'publisher_institution',
       'instituteId'],
      dtype='object')

In [30]:
def generate_graphml(edges, nodes_attrib, attrib_name="ddc"):
    graph1 = nx.convert_matrix.from_pandas_edgelist(edges, 'source','target', create_using=nx.DiGraph())
    graph1.remove_edges_from(nx.selfloop_edges(graph1))
    cycle=list(nx.simple_cycles(graph1))
    graph1.remove_edges_from(cycle)
    nx.set_node_attributes(graph1, values = nodes_attrib, name=attrib_name)
    components1 = list(nx.weakly_connected_components(graph1))
    components1.sort(key=len, reverse=True)
    g_and_eq_10  = [c for c in components1 if len(c)>=10]
    nodes_10 = [node for c in g_and_eq_10 for node in c]
    sub_graph_geq10 = graph1.subgraph(nodes_10)
    nx.write_graphml_lxml(sub_graph_geq10, f"subgraph_geq10_{attrib_name}.graphml")
    print("Done")
    return

In [31]:
generate_graphml(edges, nodes_ddc)

Done
